# Quantizing a model with ONNX Runtime for text classification tasks

This notebook shows how to apply different post-training quantization approaches such as static and dynamic quantization using [ONNX Runtime](https://onnxruntime.ai), for any tasks of the GLUE benchmark. This is made possible thanks to 🤗 [Optimum](https://github.com/huggingface/optimum), an extension of 🤗 [Transformers](https://github.com/huggingface/transformers), providing a set of performance optimization tools enabling maximum efficiency to train and run models on targeted hardwares.  

If you're opening this Notebook on colab, you will probably need to install 🤗 Transformers, 🤗 Datasets and 🤗 Optimum. Uncomment the following cell and run it.

In [1]:
#! pip install datasets transformers optimum[onnxruntime]

The GLUE Benchmark is a group of nine classification tasks on sentences or pairs of sentences which are:

- [CoLA](https://nyu-mll.github.io/CoLA/) (Corpus of Linguistic Acceptability) Determine if a sentence is grammatically correct or not.
- [MNLI](https://arxiv.org/abs/1704.05426) (Multi-Genre Natural Language Inference) Determine if a sentence entails, contradicts or is unrelated to a given hypothesis. This dataset has two versions, one with the validation and test set coming from the same distribution, another called mismatched where the validation and test use out-of-domain data.
- [MRPC](https://www.microsoft.com/en-us/download/details.aspx?id=52398) (Microsoft Research Paraphrase Corpus) Determine if two sentences are paraphrases from one another or not.
- [QNLI](https://rajpurkar.github.io/SQuAD-explorer/) (Question-answering Natural Language Inference) Determine if the answer to a question is in the second sentence or not. This dataset is built from the SQuAD dataset.
- [QQP](https://data.quora.com/First-Quora-Dataset-Release-Question-Pairs) (Quora Question Pairs2) Determine if two questions are semantically equivalent or not.
- [RTE](https://aclweb.org/aclwiki/Recognizing_Textual_Entailment) (Recognizing Textual Entailment) Determine if a sentence entails a given hypothesis or not.
- [SST-2](https://nlp.stanford.edu/sentiment/index.html) (Stanford Sentiment Treebank) Determine if the sentence has a positive or negative sentiment.
- [STS-B](http://ixa2.si.ehu.es/stswiki/index.php/STSbenchmark) (Semantic Textual Similarity Benchmark) Determine the similarity of two sentences with a score from 1 to 5.
- [WNLI](https://cs.nyu.edu/faculty/davise/papers/WinogradSchemas/WS.html) (Winograd Natural Language Inference) Determine if a sentence with an anonymous pronoun and a sentence with this pronoun replaced are entailed or not. This dataset is built from the Winograd Schema Challenge dataset.

We will see how to apply post-training static quantization on a DistilBERT model fine-tuned on the SST-2 task:

In [2]:
GLUE_TASKS = ["cola", "mnli", "mnli-mm", "mrpc", "qnli", "qqp", "rte", "sst2", "stsb", "wnli"]
task = "sst2"
model_checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
batch_size = 8

We can set our `quantization_approach` to either `dynamic` or `static` in order to apply respectively post-training dynamic and static quantization. 
- Post-training static quantization : introduces an additional calibration step where data is fed through the network in order to compute the activations quantization parameters.
- Post-training dynamic quantization : dynamically computes activations quantization parameters based on the data observed at runtime. 

In [3]:
QUANTIZATION_APPROACH = ["dynamic", "static"]

quantization_approach = "static"

* We set `per_channel` to `True` in order to apply per-channel quantization on the weights. As opposed to per-tensor quantization, which introduces one set of quantization parameters per tensor, per-channel quantization means that there will be one set of quantization parameters per channel.
* We set the quantization data type of the weights `weight_type` to `uint8`. 
* The quantization data type of the activations is set by default to `uint8` as OnnxRuntime CPU only supports activation of that type.
* We set the maximum number of examples `max_calib_samples` to use for the calibration step resulting from static quantization to `40`.
* We specify the types of operators we want to quantize in the list `op_types_to_quantize`.

In [4]:
per_channel = True
weight_type = "uint8"
max_calib_samples = 40
op_types_to_quantize = ["MatMul", "Mul", "Reshape", "Gather", "Transpose"]

## Loading the dataset

We will use the [🤗 Datasets](https://github.com/huggingface/datasets) library to download the data and get the metric we need to use for evaluation (to compare our quantized model to the baseline). This can be easily done with the functions `load_dataset` and `load_metric`.  

In [5]:
from datasets import load_dataset, load_metric

Apart from `mnli-mm` being a special code, we can directly pass our task name to those functions. `load_dataset` will cache the dataset to avoid downloading it again the next time you run this cell.

In [6]:
actual_task = "mnli" if task == "mnli-mm" else task
dataset = load_dataset("glue", actual_task)
metric = load_metric("glue", actual_task)

Reusing dataset glue (/home/ella/.cache/huggingface/datasets/glue/sst2/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad)


  0%|          | 0/3 [00:00<?, ?it/s]

Note that `load_metric` has loaded the proper metric associated to your task, which is:

- for CoLA: [Matthews Correlation Coefficient](https://en.wikipedia.org/wiki/Matthews_correlation_coefficient)
- for MNLI (matched or mismatched): Accuracy
- for MRPC: Accuracy and [F1 score](https://en.wikipedia.org/wiki/F1_score)
- for QNLI: Accuracy
- for QQP: Accuracy and [F1 score](https://en.wikipedia.org/wiki/F1_score)
- for RTE: Accuracy
- for SST-2: Accuracy
- for STS-B: [Pearson Correlation Coefficient](https://en.wikipedia.org/wiki/Pearson_correlation_coefficient) and [Spearman's_Rank_Correlation_Coefficient](https://en.wikipedia.org/wiki/Spearman%27s_rank_correlation_coefficient)
- for WNLI: Accuracy

so the metric object only computes the one(s) needed for your task.

## Preprocessing the data

Before we can feed those texts to our model, we need to preprocess them. This is done by a 🤗 Transformers `Tokenizer` which will (as the name indicates) tokenize the inputs (including converting the tokens to their corresponding IDs in the pretrained vocabulary) and put it in a format the model expects, as well as generate the other inputs that model requires.

To do all of this, we instantiate our tokenizer with the `AutoTokenizer.from_pretrained` method, which will ensure that:

- we get a tokenizer that corresponds to the model architecture we want to use
- we download the vocabulary used when pretraining this specific checkpoint

That vocabulary will be cached, so it's not downloaded again the next time we run the cell.

In [7]:
from transformers import AutoTokenizer
    
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

To preprocess our dataset, we will thus need the names of the columns containing the sentence(s). The following dictionary keeps track of the correspondence task to column names:

In [8]:
task_to_keys = {
    "cola": ("sentence", None),
    "mnli": ("premise", "hypothesis"),
    "mnli-mm": ("premise", "hypothesis"),
    "mrpc": ("sentence1", "sentence2"),
    "qnli": ("question", "sentence"),
    "qqp": ("question1", "question2"),
    "rte": ("sentence1", "sentence2"),
    "sst2": ("sentence", None),
    "stsb": ("sentence1", "sentence2"),
    "wnli": ("sentence1", "sentence2"),
}

We can then write the function that will preprocess our samples. We just feed them to the `tokenizer` with the argument `truncation=True`. This will ensure that an input longer than what the model selected can handle will be truncated to the maximum length accepted by the model.

In [9]:
sentence1_key, sentence2_key = task_to_keys[task]
max_seq_length = min(128, tokenizer.model_max_length)
padding = "max_length"

def preprocess_function(examples):
    args = (
        (examples[sentence1_key],) if sentence2_key is None else (examples[sentence1_key], examples[sentence2_key])
    )
    return tokenizer(*args, padding=padding, max_length=max_seq_length, truncation=True)

To apply this function on all the sentences (or pairs of sentences) in our dataset, we just use the `map` method of our `dataset` object we created earlier. This will apply the function on all the elements of all the splits in `dataset`, so our training, validation and testing data will be preprocessed in one single command.

In [10]:
encoded_dataset = dataset.map(preprocess_function, batched=True)

  0%|          | 0/2 [00:00<?, ?ba/s]

Even better, the results are automatically cached by the 🤗 Datasets library to avoid spending time on this step the next time you run your notebook. The 🤗 Datasets library is normally smart enough to detect when the function you pass to map has changed (and thus requires to not use the cache data). For instance, it will properly detect if you change the task in the first cell and rerun the notebook. 🤗 Datasets warns you when it uses cached files, you can pass `load_from_cache_file=False` in the call to `map` to not use the cached files and force the preprocessing to be applied again.

Note that we passed `batched=True` to encode the texts by batches together. This is to leverage the full benefit of the fast tokenizer we loaded earlier, which will use multi-threading to treat the texts in a batch concurrently.

## Applying quantization on the model

We will use the [🤗 Optimum](https://github.com/huggingface/optimum) library to instantiate an `ORTQuantizer`, which will take care of the quantization process.

To instantiate an `ORTQuantizer`, we first need to create an `ORTConfig` instance, the configuration handling the ONNX Runtime quantization related parameters.

In [11]:
from optimum.onnxruntime import ORTConfig, ORTModel, ORTQuantizer
    
ort_config = ORTConfig(
    quantization_approach=quantization_approach,
    per_channel=per_channel,
    weight_type=weight_type,
    activation_type="uint8",
    max_samples=max_calib_samples,
    calib_batch_size=batch_size,
    op_types_to_quantize=op_types_to_quantize,
)

We then create an output directory where the resulting quantized model will be saved.

The last thing to define for our `ORTQuantizer` is a calibration dataset `calib_dataset`, that will be used during the static quantization calibration step.

In [12]:
import os

model_name = model_checkpoint.split("/")[-1]
output_dir = f"{model_name}-finetuned-{task}"
os.makedirs(output_dir, exist_ok=True)

calib_dataset = encoded_dataset["train"].select(range(max_calib_samples)) if quantization_approach == "static" else None
quantizer = ORTQuantizer(ort_config, calib_dataset=calib_dataset)
quantizer.fit(model_checkpoint, output_dir, feature="sequence-classification")

To evaluate our resulting quantized model we need to define how to compute the metrics from the predictions. We need to define a function for this, which will just use the `metric` we loaded earlier, the only preprocessing we have to do is to take the argmax of our predicted logits (our just squeeze the last axis in the case of STS-B).

The metric chosen to evaluate the quantized model's performance will be Matthews correlation coefficient (MCC) for CoLA, Pearson correlation coefficient (PCC) for STS-B and accuracy for any other tasks.

In [13]:
import numpy as np

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    if task != "stsb":
        predictions = np.argmax(predictions, axis=1)
    else:
        predictions = predictions[:, 0]
    return metric.compute(predictions=predictions, references=labels)

metric_name = "pearson" if task == "stsb" else "matthews_correlation" if task == "cola" else "accuracy"

Finally, to estimate the drop in performance resulting from quantization, we are going to perform an evaluation step for both models (before and after quantization). In order to perform the latter, we will need to instantiate an `ORTModel` and thus need:

* The path of the model to evaluate.
* The dataset to use for the evaluation step.
* The model's ONNX configuration `onnx_config` associated to the model. This instance of  `OnnxConfig` describes how to export the model through the ONNX format.
* The function that will be used to compute metrics at evaluation that we defined previously `compute_metrics`.

In [14]:
from torch.utils.data import DataLoader
from transformers import default_data_collator

model_path = os.path.join(output_dir, "model.onnx")
q8_model_path = os.path.join(output_dir, "model-quantized.onnx")
onnx_config = quantizer.onnx_config
validation_key = "validation_mismatched" if task == "mnli-mm" else "validation_matched" if task == "mnli" else "validation"
eval_dataset = encoded_dataset[validation_key] 
eval_dataloader = DataLoader(eval_dataset, batch_size=batch_size, collate_fn=default_data_collator)

q8_ort_model = ORTModel(q8_model_path, onnx_config, compute_metrics=compute_metrics)
q8_model_output = q8_ort_model.evaluation_loop(eval_dataloader)
q8_model_result = q8_model_output.metrics.get(metric_name)

ort_model = ORTModel(model_path, onnx_config, compute_metrics=compute_metrics)
model_output = ort_model.evaluation_loop(eval_dataloader)
fp_model_result = model_output.metrics.get(metric_name)

ort_config.save_pretrained(output_dir)
print(f"Optimized model with {metric_name} of {round(q8_model_result * 100, 2)} saved to: {q8_model_path}.")
print(
    f"This results in a drop of {round((fp_model_result - q8_model_result) * 100, 2)} in {metric_name} when"
    f" compared to the full-precision model which has an {metric_name} of {round(fp_model_result * 100, 2)}."
)

Optimized model with accuracy of 90.6 saved to: distilbert-base-uncased-finetuned-sst-2-english-finetuned-sst2/model-quantized.onnx.
This results in a drop of 0.46 in accuracy when compared to the full-precision model which has an accuracy of 91.06.


Now let's compute the reduction in the model size resulting from quantization.

In [15]:
fp_model_size = os.path.getsize(model_path) / (1024*1024)
q_model_size = os.path.getsize(q8_model_path) / (1024*1024)

print(f"The full-precision model size is {round(fp_model_size)} MB while the quantized model one is {round(q_model_size)} MB.")
print(f"The resulting quantized model is {round(fp_model_size / q_model_size, 2)}x smaller than the full-precision one.")

The full-precision model size is 255 MB while the quantized model one is 64 MB.
The resulting quantized model is 3.97x smaller than the full-precision one.
